# Magic Personality Matcher - Preprocesamiento de datos

Para evitar tener que preprocesar los datos cada vez que abrimos el Google Colab vamos a realizar tanto particionado de los datos como su preprocesamiento en este notebook aparte.

## Fase 1 - Cargar el dataset

Lo primero es cargar el dataset con Pandas y seleccionar la cantidad de filas que queremos usar:

In [1]:
import pandas as pd

raw_data = pd.read_csv("dataset_definitivo.csv")
print("general counts")
print(raw_data['personality'].value_counts())
training_data = raw_data

print("used dataset info")
print(training_data.shape)
print(training_data.head())
print(training_data['personality'].value_counts())

general counts
personality
7     160219
3     134612
4     103804
8     103124
11     95476
12     77502
15     68449
2      46803
6      46244
5      41960
16     37019
1      33532
10     23052
14     13500
9      12765
13     11361
Name: count, dtype: int64
used dataset info
(1009422, 2)
   personality                                               post
0            4  @Pericles216 @HierBeforeTheAC @Sachinettiyil T...
1            4  @HierBeforeTheAC @Pericles216 @Sachinettiyil A...
2            4  @HierBeforeTheAC @Pericles216 @Sachinettiyil Y...
3            4  @HierBeforeTheAC @Pericles216 @Sachinettiyil Y...
4            4  @HierBeforeTheAC @Pericles216 @Sachinettiyil T...
personality
7     160219
3     134612
4     103804
8     103124
11     95476
12     77502
15     68449
2      46803
6      46244
5      41960
16     37019
1      33532
10     23052
14     13500
9      12765
13     11361
Name: count, dtype: int64


## Fase 2 - Filtrado de Stopwords, Tokenización y Stemming

El preprocesamiento de texto se realiza usando las stopwords y el tokenizer de la librería de lenguaje natural de Python. Primero definimos las stopwords, que serán las de la librería de Python y las personalidades Myers-Briggs para evitar introducir sesgos, como que, si se menciona una personalidad, que esta no se tenga en cuenta para decidir la del autor. Luego se usa el PorterStemmer para obtener la “raíz” de las palabras. Entonces el resultado es una nueva columna que contiene los posts preprocesados.

In [3]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from tqdm.contrib.concurrent import process_map
from multiprocessing import cpu_count
import re

import nltk
nltk.download('punkt')
nltk.download('stopwords')

ps = PorterStemmer()


STOPS = set(stopwords.words("english"))
CUSTOM_STOPS=set(['istj', 'isfj', 'infj', 'intj', 'istp', 'isfp', 'infp', 'intp', 'estp', 'esfp', 'enfp', 'entp', 'estj', 'esfj', 'enfj', 'entj'])

def process_text(post):
    try:
        post = post.lower()
        post = re.sub('https?://[^\s<>"]+|www\.[^\s<>"]+',' ',post)
        post = re.sub('[^0-9a-z]',' ',post)
        return " ".join([ps.stem(w) for w in word_tokenize(post) if not w in STOPS and w not in CUSTOM_STOPS])
    except:
        print("problem with: ", post)

preprocessedData = training_data.loc[:]

num_processes = cpu_count()

preprocessedData['processed_text'] = process_map(process_text, training_data['post'], max_workers=num_processes, chunksize=10)

preprocessedData

[nltk_data] Downloading package punkt to /home/mario/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/mario/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


  0%|          | 0/1009422 [00:00<?, ?it/s]

,personality,post,processed_text
0,4,@Pericles216 @HierBeforeTheAC @Sachinettiyil T...,pericles216 hierbeforetheac sachinettiyil pope...
1,4,@HierBeforeTheAC @Pericles216 @Sachinettiyil A...,hierbeforetheac pericles216 sachinettiyil perp...
2,4,@HierBeforeTheAC @Pericles216 @Sachinettiyil Y...,hierbeforetheac pericles216 sachinettiyil open...
3,4,@HierBeforeTheAC @Pericles216 @Sachinettiyil Y...,hierbeforetheac pericles216 sachinettiyil know...
4,4,@HierBeforeTheAC @Pericles216 @Sachinettiyil T...,hierbeforetheac pericles216 sachinettiyil like...
...,...,...,...
1009417,5,i loved the whole mcyt/geoguesser fandom colli...,love whole mcyt geoguess fandom collis
1009418,5,@paytonstyleslol ILL BE THINKING REAL HARD,paytonstyleslol ill think real hard
1009419,5,i am at olive garden and my sister ordered the...,oliv garden sister order 5 chees someth withou...
1009420,5,@paytonstyleslol you’re so right,paytonstyleslol right


## Fase 3 - Particionar el dataset para entrenamiento y validación
Se divide el dataset en 2 para tener 80% datos de entrenamiento y 20% de prueba

In [4]:
from sklearn.model_selection import train_test_split

X = preprocessedData['processed_text']
Y = preprocessedData['personality']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

## Fase 4 - Bolsa de palabras

Se crea una bolsa de palabras utilizando TfidVectorizer, que se basa en la frecuencia de las palabras para determinar su importancia.

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Datos de entrenamiento
bagOfWordsModel = TfidfVectorizer()
train_textsBoW = bagOfWordsModel.fit_transform(X_train)
print("X_train bag of words:")
print(train_textsBoW.shape)

# Datos pruebas
test_textsBoW = bagOfWordsModel.transform(X_test)
print("X_test bag of words:")
print(test_textsBoW.shape)

X_train bag of words:
(807537, 334414)
X_test bag of words:
(201885, 334414)


## Fase 5 - Guardar el resultado

Utilizamos joblib para guardar estas bolsas de palabras para poder usarlas para entrenamiento sin tener que repetir el proceso.

In [6]:
from tqdm.contrib.concurrent import process_map,thread_map
from joblib import dump

def multiargs_dump(args_tuple):
    dump(args_tuple[0], args_tuple[1])


thread_map(multiargs_dump,
    (
        (train_textsBoW, "X_train.lzma"),
        (test_textsBoW, "X_test.lzma"),
        (y_train, "y_train.lzma"),
        (y_test, "y_test.lzma"),
        (bagOfWordsModel, "bag_of_words.lzma")
    ))


  0%|          | 0/5 [00:00<?, ?it/s]

[None, None, None, None, None]